In [ ]:
from bs4 import BeautifulSoup
import time
import re
import pandas as pd
import json
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.action_chains import ActionChains

In [ ]:
PATH = b"C:\Users\zzbin\Desktop\Google Chrome\chromedriver.exe"
service = Service(PATH)
driver = webdriver.Chrome(service=service)

driver.maximize_window()
driver.get('https://www.transfermarkt.world/wettbewerbe/europa')
try:
    driver.switch_to.frame(driver.find_elements(By.TAG_NAME, value='iframe')[1])
    driver.find_element(By.XPATH, value='/html/body/div/div[2]/div[3]/div[2]/button').click()
except (NoSuchElementException, IndexError):
    pass

def scroll():
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
    ActionChains(driver).move_to_element(driver.find_element(By.CSS_SELECTOR, value='#yw1 > div.pager > ul')).perform()
    

def get_ligues_links():
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
    trs = soup.find_all('tr', {'class': lambda x: x in ('odd', 'even')})

    links = []
    for tr in trs:
        td = tr.find('td')
        table = td.find('table')
        tbody = table.find('tbody')
        td_link = tbody.find('tr').find_all('td')[1].find('a')
        links.append('https://www.transfermarkt.world/' +  td_link.text + '/' + '/'.join(td_link['href'].split('/')[2:]))    
    return links

scroll()

all_leagues_links = []
for i in range(2):
    
    all_leagues_links.extend(get_ligues_links())
    try:
        next_link = driver.find_element(
            By.CSS_SELECTOR,
            value='#yw1 > div.pager > ul > li.tm-pagination__list-item.tm-pagination__list-item--icon-next-page > a')
        next_link.click()
        time.sleep(3)
    
    except NoSuchElementException:
        break
    scroll()   
        
        
def get_club_links(league_link):
    driver.get(league_link)
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')        
    trs = soup.find_all('tr', {'class': lambda x: x in ('odd', 'even')})
    tds = list(map(lambda x: x.find_all('td')[1], trs))
    links = list(
        filter(lambda x: 'verein' in x, 
            list(map(lambda x: 'https://www.transfermarkt.world' + x.find('a', {'href': lambda x: x != '#'})['href'],
                     tds))))
    return links

all_club_links = []
for league_link in all_leagues_links:
    all_club_links.extend(get_club_links(league_link))      
        
players_info = {
    'Name': [],
    'Position': [],
    'Age': [],
    'Nation': [],
    'Height': [],
    'Main leg': [], 
    'Start contract date': [], 
    'Expiration contract date': [], 
    'Cost': []
    }        


def get_club_info(club_link):
    global players_info
    driver.get(club_link)
    driver.find_element(By.XPATH, value='/html/body/div[2]/div[6]/div[1]/div[1]/div[3]/a[2]/div/span').click()
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
    trs = soup.find_all('tr', {'class': lambda x: x in ('odd', 'even')})
    pattern1 = re.compile(r'.+\((\d+)\)')
    pattern2 = re.compile(r'\s+')
    for tr in trs:
        tds = tr.find_all('td')
        tbody = tds[1].find('table').find('tbody')
        tbody_trs = tbody.find_all('tr')
        players_info['Name'].append(re.sub(pattern1, ' ', tbody_trs[0].find_all('td')[1].text).strip())
        players_info['Position'].append(re.sub(pattern2, ' ', tbody_trs[1].find('td').text).strip())
        try:
            players_info['Age'].append(int(re.match(pattern1, tds[5].text).group(1)))
        except AttributeError:
            players_info['Age'].append('-')
        players_info['Nation'].append([i['title'] for i in tds[6].find_all('img')])
        players_info['Height'].append(tds[7].text)
        players_info['Main leg'].append('L' if tds[8].text == 'левая' else 'R')
        players_info['Start contract date'].append(tds[9].text)
        players_info['Expiration contract date'].append(tds[11].text)
        players_info['Cost'].append(tds[12].text)
    

In [ ]:
for club_link in all_club_links:
    get_club_info(club_link)

In [ ]:
df_info = pd.DataFrame(players_info)
df_info.to_excel('Data.xlsx')

In [ ]:
with open('data.json', 'w+') as f:
    f.write(json.dumps(players_info))